In [10]:
import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

In [11]:
def to_img(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [12]:
num_epochs = 10
batch_size = 100
learning_rate = 1e-3

In [13]:
img_transform = transforms.Compose([
    transforms.ToTensor()
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [14]:
dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [15]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        #return F.sigmoid(self.fc4(h3))
        return self.fc4(h3)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar
    
    

In [1]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(784, 400),
            nn.ReLU(),
            nn.Linear(400, 20),
            nn.ReLU(),
            nn.Linear(200, 100),
            nn.ReLU(),
            nn.Linear(100, 20)
        )
        
        self.em = nn.Linear(20,latent_dim)  # mu layer
        self.ev = nn.Linear(20,latent_dim)  # logvariance layer
        
    def encode(self, x):
        #h1 = F.relu(self.fc1(x))
        h1 = self.encoder(x)
        mu = self.em(h1)
        logvar = self.ev(out)
        
        return mu,logvar
        
        #return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        #return F.sigmoid(self.fc4(h3))
        return self.fc4(h3)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar

NameError: name 'nn' is not defined

In [25]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(dataloader):
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        if torch.cuda.is_available():
            img = img.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(img)
        loss = loss_function(recon_batch, img, mu, logvar)
        loss.backward()
        train_loss += loss.data
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(img),
                len(dataloader.dataset), 100. * batch_idx / len(dataloader),
                loss.data / len(img)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(dataloader.dataset)))
    #if epoch % 10 == 0:
    #    save = to_img(recon_batch.cpu().data)
    #    save_image(save, './vae_img/image_{}.png'.format(epoch))

#torch.save(model.state_dict(), './vae.pth')

/Users/sushanta/anaconda3/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Train Epoch: 0 [0/60000 (0%)]	Loss: 32.535355
Train Epoch: 0 [10000/60000 (17%)]	Loss: 31.073637
Train Epoch: 0 [20000/60000 (33%)]	Loss: 31.753145
Train Epoch: 0 [30000/60000 (50%)]	Loss: 31.344830
Train Epoch: 0 [40000/60000 (67%)]	Loss: 32.467789
Train Epoch: 0 [50000/60000 (83%)]	Loss: 31.262062
====> Epoch: 0 Average loss: 31.6393
Train Epoch: 1 [0/60000 (0%)]	Loss: 31.824566
Train Epoch: 1 [10000/60000 (17%)]	Loss: 31.354553
Train Epoch: 1 [20000/60000 (33%)]	Loss: 30.875444
Train Epoch: 1 [30000/60000 (50%)]	Loss: 30.997263
Train Epoch: 1 [40000/60000 (67%)]	Loss: 32.286819
Train Epoch: 1 [50000/60000 (83%)]	Loss: 32.306030
====> Epoch: 1 Average loss: 31.4908
Train Epoch: 2 [0/60000 (0%)]	Loss: 32.925915
Train Epoch: 2 [10000/60000 (17%)]	Loss: 30.701485
Train Epoch: 2 [20000/60000 (33%)]	Loss: 31.073057
Train Epoch: 2 [30000/60000 (50%)]	Loss: 30.259224
Train Epoch: 2 [40000/60000 (67%)]	Loss: 32.334541
Train Epoch: 2 [50000/60000 (83%)]	Loss: 31.176298
====> Epoch: 2 Average 

In [16]:
model = VAE()
if torch.cuda.is_available():
    model.cuda()

In [17]:
reconstruction_function = nn.MSELoss(size_average=False)

/Users/sushanta/anaconda3/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [18]:
def loss_function(recon_x, x, mu, logvar):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    BCE = reconstruction_function(recon_x, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    # KL divergence
    return BCE + KLD


def vae_loss(x, x_hat, mu, logvar):
    reconstruction_function = nn.MSELoss(size_average=False)
    BCE = reconstruction_function(x_hat, x)
    KLD = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD).mul_(-0.5)
    return BCE + KLD

In [19]:
def loss_function(x_hat, x, mu, logvar):
    reconstruction_function = nn.MSELoss(size_average=False)
    BCE = reconstruction_function(x_hat, x)
    KLD = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD).mul_(-0.5)
    return BCE + KLD

In [20]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [26]:
for batch_idx, data in enumerate(dataloader):
    break;
img, _ = data
img = img.view(img.size(0), -1)
img = Variable(img)

recon_batch, mu, logvar = model(img)

#img
save_before=to_img(img.cpu().data)
save_image(save_before, 'image_before{}.png'.format(0))


optimizer.zero_grad()
recon_batch, mu, logvar = model(img)

In [27]:
#recon_batch
save_after = to_img(recon_batch.cpu().data)
save_image(save_after, 'image_after{}.png'.format(0))

In [21]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(dataloader):
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        if torch.cuda.is_available():
            img = img.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(img)
        loss = loss_function(recon_batch, img, mu, logvar)
        loss.backward()
        train_loss += loss.data
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(img),
                len(dataloader.dataset), 100. * batch_idx / len(dataloader),
                loss.data / len(img)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(dataloader.dataset)))
    #if epoch % 10 == 0:
    #    save = to_img(recon_batch.cpu().data)
    #    save_image(save, './vae_img/image_{}.png'.format(epoch))

#torch.save(model.state_dict(), './vae.pth')

/Users/sushanta/anaconda3/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Train Epoch: 0 [0/60000 (0%)]	Loss: 144.079910
Train Epoch: 0 [10000/60000 (17%)]	Loss: 44.894394
Train Epoch: 0 [20000/60000 (33%)]	Loss: 40.300987
Train Epoch: 0 [30000/60000 (50%)]	Loss: 39.239471
Train Epoch: 0 [40000/60000 (67%)]	Loss: 38.097588
Train Epoch: 0 [50000/60000 (83%)]	Loss: 35.484856
====> Epoch: 0 Average loss: 41.2877
Train Epoch: 1 [0/60000 (0%)]	Loss: 35.987114
Train Epoch: 1 [10000/60000 (17%)]	Loss: 35.835648
Train Epoch: 1 [20000/60000 (33%)]	Loss: 35.318691
Train Epoch: 1 [30000/60000 (50%)]	Loss: 35.463200
Train Epoch: 1 [40000/60000 (67%)]	Loss: 34.669506
Train Epoch: 1 [50000/60000 (83%)]	Loss: 36.320511
====> Epoch: 1 Average loss: 34.9482
Train Epoch: 2 [0/60000 (0%)]	Loss: 33.321186
Train Epoch: 2 [10000/60000 (17%)]	Loss: 33.118240
Train Epoch: 2 [20000/60000 (33%)]	Loss: 34.749302
Train Epoch: 2 [30000/60000 (50%)]	Loss: 33.870762
Train Epoch: 2 [40000/60000 (67%)]	Loss: 35.599121
Train Epoch: 2 [50000/60000 (83%)]	Loss: 34.677006
====> Epoch: 2 Average